In [1]:
from main_workflow import *
import pandas as pd
from tqdm import tqdm
import evaluate

/Users/nltyh/Documents/NUS/Y4/DSA4213/DSA4213-Group-Project/.venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/Users/nltyh/Documents/NUS/Y4/DSA4213/DSA4213-Group-Project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-02 01:49:14.346 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
run_travel_bot("Hi can you plan me a trip from singapore to tokyo from 10th of november 2025 to 17th november 2025, for 2 adults and 2 children, with an economy budget, flight arrival in the morning and a preference for a 3 star hotel with the most attractions nearby ")

2025-11-02 01:49:19.110 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 01:49:19.178 
  command:

    streamlit run /Users/nltyh/Documents/NUS/Y4/DSA4213/DSA4213-Group-Project/.venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-02 01:49:19.178 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 01:49:19.179 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 01:49:19.179 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 01:49:19.682 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 01:49:19.682 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.

"Flights (Round Trip)\n* Outbound (SIN → NRT)\n  Scoot, China Eastern Airlines — SIN 10 Nov 2025 23:20 → NRT 11 Nov 2025 07:10 | SGD 1225.12 | A2 C2 ECONOMY\n* Return (NRT → SIN)\n  Scoot, China Eastern Airlines — NRT 17 Nov 2025 15:20 → SIN 18 Nov 2025 11:40\n\nHotel\n* Hotel Name: Apa Nihonbashi Ekimae\n* Rating: 3★\n* Address: 3 Chome-12-14 Higashinihonbashi CHUO-KU\n* Website: http://www.apahotel.com/~language/en/hotel/nihonbashi-ekimae/index.html\n* Description: Set in a prime location of Tokyo, APA Hotel Higashi-Nihonbashi-Ekimae puts everything the city has to offer just outside your doorstep. The hotel offers a wide range of amenities to ensure a great stay, including comfortable guestrooms with plasma TVs, complimentary wireless internet, non-smoking rooms, and air conditioning. Massage services are also available.\n* Facilities: Room service, Laundry service, 24-hour security, Hotel safe, Lift access, Newspapers, Luggage room, Towels and bed linen, Bar, Restaurant, Smoking ar

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
gc = pd.read_csv('golden_copy.csv')

In [4]:
baseline_res = pd.read_csv('baseline_responses.csv')

In [5]:
results = []

for idx, row in tqdm(gc.iterrows(), total = len(gc)):
    query = row['user_query']
    res = run_travel_bot(query)
    results.append([query, res])

baseline_responses = pd.DataFrame(results, columns=['user_query', 'response'])

 59%|█████▉    | 23/39 [14:56<11:11, 41.98s/it]

 74%|███████▍  | 29/39 [19:30<07:14, 43.46s/it]

100%|██████████| 39/39 [26:04<00:00, 40.12s/it]


In [136]:
bs = pd.read_csv('baseline_responses.csv')
gc = pd.read_csv('golden_copy.csv')

In [137]:
merged_df = pd.merge(bs, gc, on='user_query', how='outer')


In [ ]:
merged_df

In [150]:
import evaluate
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /Users/nltyh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [151]:
df = merged_df[['user_query', 'response', 'golden_response']]

In [152]:
bertscore = evaluate.load('bertscore')
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /Users/nltyh/nltk_data...
[nltk_data] Downloading package punkt_tab to /Users/nltyh/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/nltyh/nltk_data...


In [ ]:
bertscore_results = bertscore.compute(
    predictions=df["response"].tolist(),
    references=df["golden_response"].tolist(),
    lang="en"
)

df["BERTScore_F1"] = bertscore_results["f1"]

# print(df[["user_query", "BERTScore_F1"]])

# Optionally print overall averages
print("\nAverage BERTScore F1:", sum(df["BERTScore_F1"]) / len(df))


Average BERTScore F1: 0.8550686163780017


In [163]:
from nltk.translate.meteor_score import single_meteor_score
from nltk.tokenize import word_tokenize

In [164]:
def compute_meteor(hypothesis: str, reference: str) -> float:
    # tokenize both hypothesis and reference into lists of words
    hyp_tokens = word_tokenize(hypothesis)
    ref_tokens = word_tokenize(reference)
    return single_meteor_score(ref_tokens, hyp_tokens)

In [166]:
compute_meteor(merged_df['response'][1], merged_df['golden_response'][1])

0.5309944710390497

In [167]:
from evals import ModelEvaluator

In [169]:
evaluator = ModelEvaluator(merged_df)

In [170]:
bertscore_df, avg_score = evaluator.bertEval()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [171]:
meteor_df, avg_meteor = evaluator.meteorEval()

In [173]:
avg_meteor

np.float64(0.425310123487645)

In [174]:
avg_score

np.float64(0.8550686163780017)